In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
from pprint import pprint

In [2]:
def querySparql(query):
    # The ld.stadt-zuerich.ch endpoints doesn't seem to work, use LINDA instead
    #sparql = SPARQLWrapper("https://ld.stadt-zuerich.ch/query")
    sparql = SPARQLWrapper("https://lindas-data.ch:8443/lindas/query")
    sparql.setHTTPAuth("BASIC")
    sparql.setCredentials("public", "public")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    return results['results']['bindings']

In [3]:
allDatasets = """
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?dataset ?label WHERE { GRAPH <https://linked.opendata.swiss/graph/zh/statistics> {

   ?dataset a qb:DataSet ; 
      rdfs:label ?label .     
    
   # ?obs a qb:Observation ;
   ?obs <http://purl.org/linked-data/cube#dataSet> ?dataset .
   
   # FILTER (?dataset = <https://ld.stadt-zuerich.ch/statistics/dataset/AVA-RAUM-ZEIT-GGH-HEL-SEX>)  
}}
GROUP BY ?dataset ?label
LIMIT 1000
"""
print allDatasets


PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?dataset ?label WHERE { GRAPH <https://linked.opendata.swiss/graph/zh/statistics> {

   ?dataset a qb:DataSet ; 
      rdfs:label ?label .     
    
   # ?obs a qb:Observation ;
   ?obs <http://purl.org/linked-data/cube#dataSet> ?dataset .
   
   # FILTER (?dataset = <https://ld.stadt-zuerich.ch/statistics/dataset/AVA-RAUM-ZEIT-GGH-HEL-SEX>)  
}}
GROUP BY ?dataset ?label
LIMIT 1000



In [4]:
datasetResult = querySparql(allDatasets)

In [5]:
datasets = [{'uri': d['dataset']['value'], 'label': d['label']['value']} for d in datasetResult]
datasets

[{'label': u'Arbeitsst\xe4tten nach Betriebsart, Raum, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/AST-RAUM-ZEIT-BTA'},
 {'label': u'Besch\xe4ftigte nach Betriebsart, Raum, Geschlecht, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/BES-RAUM-ZEIT-BTA-SEX'},
 {'label': u'Besch\xe4ftigte nach Betriebsart, Raum, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/BES-RAUM-ZEIT-BTA'},
 {'label': u'Zuschauer/innen, Besucher/innen nach Betriebsart, Heimatland, Raum, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/ZUS-RAUM-ZEIT-BTA-HEL'},
 {'label': u'Zuschauer/innen, Besucher/innen nach Betriebsart, Raum, Geschlecht, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/ZUS-RAUM-ZEIT-BTA-SEX'},
 {'label': u'Zuschauer/innen, Besucher/innen nach Betriebsart, Raum, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/ZUS-RAUM-ZEIT-BTA'},
 {'label': u'Sch\xfcler/innen und Student/innen nach Betriebsart, Ra

In [ ]:
metadata = []
for dataset in datasets:
    metaQuery = u'''
    PREFIX qb: <http://purl.org/linked-data/cube#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?dataset ?title ?categoryLabel ?quelleLabel ?zeit ?updateDate ?glossarLabel ?btaLabel ?raumLabel
    WHERE { GRAPH <https://linked.opendata.swiss/graph/zh/statistics> {
       ?dataset a qb:DataSet ; 
            rdfs:label ?title .     

        # group
       OPTIONAL {
           ?category a <https://ld.stadt-zuerich.ch/schema/Category> ;
           rdfs:label ?categoryLabel ;
           skos:narrower* ?dataset .
        }

         # quelle, zeit, aktualisierungsdatum
        ?obs <http://purl.org/linked-data/cube#dataSet> ?dataset .
        OPTIONAL {
            ?obs <https://ld.stadt-zuerich.ch/statistics/attribute/QUELLE> ?quelle .
            ?quelle rdfs:label ?quelleLabel .
        }
        OPTIONAL {
            ?obs <https://ld.stadt-zuerich.ch/statistics/property/RAUM> ?raum .
            ?raum rdfs:label ?raumLabel .
        }
        OPTIONAL { ?obs <https://ld.stadt-zuerich.ch/statistics/property/ZEIT> ?zeit } .
        OPTIONAL { ?obs <https://ld.stadt-zuerich.ch/statistics/attribute/DATENSTAND> ?updateDate } .
        
        # evtl. GLOSSAR und BTA (und weitere) für Tags verwenden
        OPTIONAL {
            ?obs <https://ld.stadt-zuerich.ch/statistics/attribute/GLOSSAR> ?glossar .
            ?glossar rdfs:label ?glossarLabel .
        }
         OPTIONAL {
             ?obs <https://ld.stadt-zuerich.ch/statistics/property/BTA> ?bta .        
             ?bta rdfs:label ?btaLabel .
        }

       FILTER (?dataset = <DATASET_URI>)
    }}
    GROUP BY ?dataset ?title ?categoryLabel ?quelleLabel ?zeit ?updateDate ?glossarLabel ?btaLabel ?raumLabel
    LIMIT 1000'''
    metaQuery = metaQuery.replace('DATASET_URI', dataset['uri'])
    print metaQuery
    
    metaResult = querySparql(metaQuery)
    pprint(metaResult[0])
    
    meta = {
        'uri': dataset['uri'],
        'title': dataset['label'],
        'description': '',
        'category': list(set([t.get('categoryLabel', {}).get('value', '') for t in metaResult])),
        'legal_basis': '',
        'spatial': list(set([t.get('raumLabel', {}).get('value', '') for t in metaResult])),
        'publisher': '',
        'source': '',
        'temporal': '',
        'quality': '',
        'published_date': list(set([t.get('zeit', {}).get('value', '') for t in metaResult])),
        'update_date': list(set([t.get('updateDate', {}).get('value', '') for t in metaResult])),
        'data_type': '',
        'periodicity': '',
        'tags': list(set([t.get('glossarLabel', {}).get('value', '') for t in metaResult])),
        'version': '',
        'remarks': '',
    }
    metadata.append(meta)
    pprint(meta)

In [ ]:
pprint(metadata)
